In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

import plotly.express as px
from scipy import stats
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

# set plotly parameters
import plotly.io as pio
pio.renderers.default='notebook'

In [23]:
df = pd.read_csv("../../../Data/csv/1TF_BQ_Accidents.csv", header=None)
df = df.rename({19: 'date'}, axis=1)
df['date'] = pd.to_datetime(df['date'])

df = df.groupby('date').size().reset_index(name='count')

C:\Users\hanne\AppData\Local\Temp\ipykernel_15356\3011639115.py:1: DtypeWarning:

Columns (3,7,11) have mixed types. Specify dtype option on import or set low_memory=False.



In [24]:
df['dow'] = df['date'].apply(lambda x: x.weekday())
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [25]:
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [26]:
df = encode(df,'dow', 6)
df = encode(df,'month', 12)
df = encode(df,'day', 31)

In [27]:
sc=StandardScaler()
df['count'] = sc.fit_transform(df['count'].values.reshape(-1,1))

In [28]:
df.drop(columns=(['date', 'dow','day','month','year']),inplace=True)

In [29]:
def create_lookback(data, n_in=1, n_out=1):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list() 
    #input sequence (t-n, ...., t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    #forecast sequence
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)

    return agg

In [30]:
reframed = create_lookback(df,1,1)
reframed.drop(reframed.columns[[8,9,10,11,12,13]],axis=1,inplace=True)
values = reframed.values

In [31]:
X = values[:, :-1]
y = values[:, -1]
train_X, test_X,train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 0,shuffle=False)

In [32]:
train_X = train_X.reshape((train_X.shape[0],1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0],1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


(1752, 1, 7) (1752,) (438, 1, 7) (438,)


In [33]:
model = Sequential()
model.add(LSTM(15, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

c:\Users\hanne\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [34]:
history = model.fit(train_X, train_y, epochs=100, batch_size=28, validation_data=(test_X, test_y),verbose=1, shuffle=False)

Epoch 1/100


63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.6924 - val_loss: 0.7005
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6604 - val_loss: 0.6710
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6375 - val_loss: 0.6489
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6192 - val_loss: 0.6375
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6079 - val_loss: 0.6320
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6023 - val_loss: 0.6286
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5994 - val_loss: 0.6273
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5971 - val_loss: 0.6263
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5947 - val_loss: 0.6246
Epoch 10/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5924 - val_loss: 0.6228
Epoch 11/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5901 - val_loss: 0.6205
Epoch 12/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5877 - val_lo

In [35]:
y_pred = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [36]:
# invert scaling for forecast
inv_yhat = np.concatenate((y_pred, test_X[:, 1:]), axis=1)
inv_yhat = sc.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [37]:
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = sc.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [44]:
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 19.730


In [39]:
#inv_ypred = np.concatenate((y_pred, test_X[:, 1:]), axis=1)

In [46]:
chart = pd.DataFrame()
#y_pred = y_pred.ravel().tolist()
chart['predicted'] = inv_yhat
chart['actual'] = inv_y

In [47]:
fig = px.line(chart)
fig.show()